# Predictive Modelling: XGBoost

# Imports

In [64]:
%load_ext autoreload
%autoreload 2

# Pandas and numpy
import pandas as pd
import numpy as np

#
from IPython.display import display, clear_output
import sys
import time

# Libraries for Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from src.visualization.visualize import plot_corr_matrix, plot_multi, plot_norm_dist, plot_feature_importances

# Some custom tools
from src.data.tools import check_for_missing_vals

# Alpaca API
import alpaca_trade_api as tradeapi

# Pickle
import pickle
import os
from pathlib import Path

# To load variables from .env file into system environment
from dotenv import find_dotenv, load_dotenv

from atomm.Indicators import MomentumIndicators
from atomm.DataManager.main import MSDataManager
from atomm.Tools import calc_open_position
from src.visualization.visualize import plot_confusion_matrix
from atomm.Methods import BlockingTimeSeriesSplit, PurgedKFold           


import time

# scikit-learn
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, TimeSeriesSplit
from xgboost import XGBClassifier

# For BayesianHyperparameter Optimization
from src.models.hyperparameter_optimization import search_space, BayesianSearch

# Visualization libraries
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
import matplotlib.gridspec as gridspec
#import matplotlib.style as style
from scipy import stats

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Load environment variables
load_dotenv(find_dotenv())

True

## Defining functions

In [247]:
# compute moving averages
fast_window = 20
slow_window = 50
data = djia['IBM'].copy()
data['fast_mavg'] = data['Close'].rolling(window=fast_window, min_periods=1, center=False).mean()
data['slow_mavg'] = data['Close'].rolling(window=slow_window, min_periods=1, center=False).mean()
data.head()

# Compute sides
data['side'] = np.nan

long_signals = data['fast_mavg'] >= data['slow_mavg'] 
short_signals = data['fast_mavg'] < data['slow_mavg'] 
data.loc[long_signals, 'side'] = 1
data.loc[short_signals, 'side'] = -1

# Remove Look ahead biase by lagging the signal
data['side'] = data['side'].shift(1)

# Save the raw data
raw_data = data.copy()

# Drop the NaN values from our data set
#data.dropna(axis=0, how='any', inplace=True)

data['side'].value_counts()

-1.0    982
 1.0    971
Name: side, dtype: int64

In [233]:
from atomm.Filters import cusum_filter
from atomm.Tools import get_daily_vol
from atomm.Models.Labelling import add_vertical_barrier, get_events, get_bin


In [253]:
cusum

DatetimeIndex(['2012-02-03 00:00:00+00:00', '2012-02-23 00:00:00+00:00',
               '2012-03-05 00:00:00+00:00', '2012-03-06 00:00:00+00:00',
               '2012-03-08 00:00:00+00:00', '2012-03-13 00:00:00+00:00',
               '2012-03-15 00:00:00+00:00', '2012-03-26 00:00:00+00:00',
               '2012-04-02 00:00:00+00:00', '2012-04-04 00:00:00+00:00',
               ...
               '2019-09-05 00:00:00+00:00', '2019-09-10 00:00:00+00:00',
               '2019-09-16 00:00:00+00:00', '2019-09-30 00:00:00+00:00',
               '2019-10-02 00:00:00+00:00', '2019-10-08 00:00:00+00:00',
               '2019-10-10 00:00:00+00:00', '2019-10-17 00:00:00+00:00',
               '2019-10-25 00:00:00+00:00', '2019-11-04 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', length=550, freq=None)

In [252]:
vertical_barriers.head()

2012-02-03 00:00:00+00:00   2012-02-06 00:00:00+00:00
2012-02-23 00:00:00+00:00   2012-02-24 00:00:00+00:00
2012-03-05 00:00:00+00:00   2012-03-06 00:00:00+00:00
2012-03-06 00:00:00+00:00   2012-03-07 00:00:00+00:00
2012-03-08 00:00:00+00:00   2012-03-09 00:00:00+00:00
Name: Epoch, dtype: datetime64[ns, UTC]

In [249]:
daily_vol = get_daily_vol(djia['IBM'], lookback=20)

/Users/robin/Documents/temp/atomm/src/atomm/Indicators.py:169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df[f'atr_{n}'] = atr


In [243]:
cusum = cusum_filter(djia['IBM']['Close'], threshold=daily_vol)

In [232]:
vertical_barriers = add_vertical_barrier(cusum, djia['IBM']['Close'], num_days=1)

In [280]:
pt_sl = [1, 1]
min_ret = 0.005
triple_barrier_events = get_events(
    close=djia['IBM']['Close'],
    pt_sl=pt_sl,
    target=daily_vol,
    min_ret=min_ret,
    vertical_barrier_times=vertical_barriers,
    t_events=cusum,
    side_prediction=data['side'],
    num_threads=4
)

2020-01-06 17:47:44.959938 100.0% apply_pt_sl_on_t1 done after 0.01 minutes. Remaining 0.0 minutes.


In [282]:
triple_barrier_events.side.value_counts()
labels = ml.labeling.get_bins(triple_barrier_events, djia['IBM']['Close'])


 1.0    282
-1.0    268
Name: side, dtype: int64

In [196]:
# Import 
import atomm.Models.Tuning as mt
mt.search_space(SVC)

mt.BayesianSearch()

{'scale': <hyperopt.pyll.base.Apply at 0x1a3fb7aed0>,
 'normalize': <hyperopt.pyll.base.Apply at 0x1a40226410>,
 'gamma': <hyperopt.pyll.base.Apply at 0x1a401430d0>,
 'C': <hyperopt.pyll.base.Apply at 0x1a3fe19210>,
 'cv': <hyperopt.pyll.base.Apply at 0x1a401639d0>,
 'model': sklearn.svm._classes.SVC}

In [6]:
def calc_returns(signals, prices):
    returns = prices[['Close']].pct_change()
    returns['Signal_Strat'] = calc_open_position(signals)
    returns['Cum_Returns_Strat'] = (returns['Signal_Strat'] * returns['Close']).cumsum()
    returns['Cum_Returns_BH'] = returns['Close'].cumsum()
    return returns

## Loading the data

In [202]:
data_base_dir = os.environ.get('DATA_DIR_BASE_PATH')

fname = os.path.join(data_base_dir, 'processed', 'index.h5')
fname = Path(fname)
# Load dataset from HDF storage
with pd.HDFStore(fname) as storage:
    djia = storage.get('nyse/cleaned/rand_symbols')
    y_2c = storage.get('nyse/engineered/target_two_class')
    y_3c = storage.get('nyse/engineered/target_three_class')
    df_moments = storage.get('nyse/engineered/features')
    #print(storage.info())
    
# Create copies of the pristine data
X = df_moments.copy()
y = y_3c.copy()
y2 = y_2c.copy()
prices = djia.copy()

In [8]:
forecast_horizon = [1, 3, 5, 7, 10, 15, 20, 25, 30]
input_window_size = [3, 5, 7, 10, 15, 20, 25, 30]
ti_list = ['macd', 'rsi', 'stoc', 'roc', 'bbu', 'bbl', 'ema', 'atr', 'adx', 'cci', 'williamsr', 'stocd']
symbol_list = df_moments.columns.get_level_values(0).unique()

## Imputing missing values

In [9]:
X.shape

(1954, 4800)

In [10]:
check_for_missing_vals(X)

No missing values found in dataframe


Prices values

In [11]:
prices.shape

(1954, 250)

In [12]:
check_for_missing_vals(prices)

No missing values found in dataframe


In [13]:
y_3c.shape

(1954, 450)

In [14]:
check_for_missing_vals(y_3c)

No missing values found in dataframe


In [15]:
y2.shape

(1954, 450)

In [16]:
check_for_missing_vals(y2)

No missing values found in dataframe


No missing values, and sizes of ```y.shape[0]``` and```X.shape[0]``` match.

# Scaling the features

In [17]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [18]:
#scale = MinMaxScaler()
scale = StandardScaler()

In [19]:
scaled = scale.fit_transform(X)

In [20]:
scaled.shape

(1954, 4800)

In [21]:
X_scaled = pd.DataFrame(data=scaled, columns=X.columns)
#X_scaled = X

In [22]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split, TimeSeriesSplit

# Train-Test Split

In [23]:
# Use 70/30 train/test splits
test_p = .3

In [53]:
# Scaled, three-class
test_size = int((1 - test_p) * X_scaled.shape[0])
X_train, X_test, y_train, y_test = X_scaled[:test_size], X_scaled[test_size:], y_3c[:test_size], y_3c[test_size:]
prices_train, prices_test = djia[:test_size], djia[test_size:]

In [24]:
# Unscaled, two-class
test_size = int((1 - test_p) * X.shape[0])
X_train, X_test, y_train, y_test = X[:test_size], X[test_size:], y2[:test_size], y2[test_size:]
prices_train, prices_test = djia[:test_size], djia[test_size:]

In [44]:
# Scaled, two-class
test_size = int((1 - test_p) * X.shape[0])
X_train, X_test, y_train, y_test = X_scaled[:test_size], X_scaled[test_size:], y2[:test_size], y2[test_size:]
prices_train, prices_test = djia[:test_size], djia[test_size:]

In [ ]:
#test_size = test_p
#X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_3c, test_size=test_size, random_state=101)

# Model

In [157]:
symbol = 'T'
n = 15
# set up cross validation splits
tscv = TimeSeriesSplit(n_splits=5)
btscv = BlockingTimeSeriesSplit(n_splits=5)

In [185]:
# Purged KFold Cross Validation
n=10
t1 = pd.Series([y_2c.index[i+n] for i in range(len(y_2c)-n)], index=y_2c.index[:-n])
ppcv = PurgedKFold(n_splits=5, t1=t1, pctEmbargo=0.01)
split = ppcv.split(X[:-n])

In [186]:
t1[t1<=t1.index[20]].index

DatetimeIndex(['2012-02-02 00:00:00+00:00', '2012-02-03 00:00:00+00:00',
               '2012-02-06 00:00:00+00:00', '2012-02-07 00:00:00+00:00',
               '2012-02-08 00:00:00+00:00', '2012-02-09 00:00:00+00:00',
               '2012-02-10 00:00:00+00:00', '2012-02-13 00:00:00+00:00',
               '2012-02-14 00:00:00+00:00', '2012-02-15 00:00:00+00:00',
               '2012-02-16 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='Epoch', freq=None)

In [189]:
next(btscv.split(X[:-n]))


(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
        130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
        143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
        156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
        169, 170, 171, 172, 173, 174, 175, 176, 177

## Single lookback/lookahead combination

In [54]:
clf = XGBClassifier(n_jobs=-1)
clf.fit(X_train[symbol][[f'{x}_{n}' for x in ti_list]], y_train[symbol][f'signal_{n}'])

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=-1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [55]:
y_pred_svc = clf.predict(X_test[symbol][[f'{x}_{n}' for x in ti_list]])
print(classification_report(y_test[symbol][f'signal_{n}'], y_pred_svc))
print(confusion_matrix(y_test[symbol][f'signal_{n}'], y_pred_svc))
#plot_feature_importances(svc.coef_,
#                         X_scaled[symbol][[f'{x}_{n}' for x in ti_list]].columns, 
#                         model='SVC1', top_count=100)

              precision    recall  f1-score   support

          -1       0.39      0.06      0.10       155
           0       0.42      0.60      0.49       244
           1       0.37      0.42      0.40       188

    accuracy                           0.40       587
   macro avg       0.39      0.36      0.33       587
weighted avg       0.40      0.40      0.36       587

[[  9 105  41]
 [  5 147  92]
 [  9 100  79]]


## All combinations

In [62]:
clf = XGBClassifier
score_xgb, returns_xgb = run_combinations(
    symbol=symbol,
    forecast_horizon=forecast_horizon,
    input_window_size=input_window_size,
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
    prices=prices_test,
    model=clf,
    hyper_optimize=True,
    n_eval=5
)

plot_corr_matrix(score_xgb, mask_upper=False, show_annot=True, figsize=(8, 8))

 60%|██████    | 3/5 [00:02<00:01,  1.25it/s, best loss: -0.5381818181818182]


SystemExit: 0

//anaconda3/envs/capstone_nf/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3327: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Averaging across all 50 randomly selected stocks

In [172]:
avg, _, _ = avg_model(
    symbol_list,
    forecast_horizon,                                 
    input_window_size,                                  
    X_train,    
    X_test,    
    y_train,    
    y_test,    
    prices_test,
    model=clf,
    silent = False
)

'Leg [50/50]; Elapsed Time 337.0s\n'

,3,5,7,10,15,20,25,30
1,0.413629,0.41339,0.411721,0.412368,0.409506,0.413663,0.414037,0.412129
3,0.406065,0.406814,0.409165,0.407836,0.406951,0.410971,0.411959,0.414514
5,0.402078,0.405486,0.406917,0.405077,0.408893,0.407939,0.406337,0.408382
7,0.40644,0.406746,0.40586,0.402351,0.406678,0.40954,0.406303,0.413799
10,0.407836,0.407359,0.407019,0.40385,0.408041,0.409915,0.408825,0.405451
15,0.402589,0.408484,0.400818,0.402249,0.405554,0.406269,0.39816,0.397683
20,0.396491,0.398705,0.398637,0.394719,0.397922,0.397002,0.39925,0.396559
25,0.397615,0.394617,0.398126,0.396934,0.400443,0.401193,0.399761,0.396899
30,0.396048,0.394991,0.395707,0.392811,0.397683,0.406269,0.401704,0.398535


## Hyperparamter Optimization: GridSearch

In [ ]:
start = time.time()

# kernel = 'rbf', 'linear, 'poly'
# gamma only for non-linear kernels (poly, rbf)
# C penalty for error term
# degrees = [0, 1, 2, 3, 4, 5, 6] only for kernel = poly
gammas = [10, 1, 0.1, 0.01, 0.001, 1E-4, 1E-5, 1E-6]
cs = [1, 10, 100, 1000, 1E4, 1E5, 1E6]
param_search = [
    {'kernel': ['rbf'], 'gamma': gammas, 'C': cs},
    #{'kernel': ['poly'], 'gamma': gammas, 'C': cs, 'degree' : [0, 1, 2, 3, 4, 5, 6]},
    #{'kernel': ['linear'], 'C': cs}
]

xgb_gs = XGBClassifier()
gsearch_xgb = GridSearchCV(
    estimator=xgb_gs,                   
    cv=btscv.split(X_train[symbol][[f'{x}_{n}' for x in ti_list]]),      
    param_grid=param_search,                      
    scoring = 'accuracy',
    n_jobs=-1
)
gsearch_xgb.fit(X_train[symbol][[f'{x}_{n}' for x in ti_list]], y_train[symbol][f'signal_{n}'])
print(f'Elapsed time: {round(time.time()-start, 0)}s')

In [91]:
estimator = gsearch_svc.best_estimator_
cvs = cross_val_score(
    estimator, 
    X_train[symbol][[f'{x}_{n}' for x in ti_list]], 
    y_train[symbol][f'signal_{n}'], 
    cv=btscv.split(X_train[symbol][[f'{x}_{n}' for x in ti_list]])
)
results = pd.DataFrame(gsearch_xgb.cv_results_)
print('##### Results #####')
print('Score best parameters: ', gsearch_xgb.best_score_)
print('Best parameters: ', gsearch_xgb.best_params_)
print('Cross-validation Score: ', cvs.mean())
print('Test Score: ', estimator.score(X_test[symbol][[f'{x}_{n}' for x in ti_list]], y_test[symbol][f'signal_{n}']))
print('Parameter combinations evaluated: ', results.shape[0])

NameError: name 'gsearch_svc' is not defined

In [163]:
gsearch_xgb.best_score_

0.5876651982378854

## Hyperparamter Optimization: Bayesian Optimization

In [56]:
n=15
symbol='T'

model = XGBClassifier
bsearch_xgb = BayesianSearch(
    search_space(XGBClassifier),
    model,
    X_train[symbol][[f'{x}_{n}' for x in ti_list]], 
    y_train[symbol][f'signal_{n}'], 
    X_test[symbol][[f'{x}_{n}' for x in ti_list]],
    y_test[symbol][f'signal_{n}'],
    num_eval=25
)

100%|██████████| 10/10 [06:43<00:00, 40.35s/it, best loss: -0.5449907756477098]
##### Results #####
Score best parameters:  -0.5449907756477098
Best parameters:  {'booster': 'gblinear', 'colsample_bytree': 0.8962667825056397, 'cv': 'cv', 'gamma': 0, 'learning_rate': 0.01399485984960853, 'max_depth': 144, 'model': <class 'xgboost.sklearn.XGBClassifier'>, 'n_estimators': 984, 'n_jobs': -1, 'subsample': 1}
Test Score:  0.41567291311754684
Parameter combinations evaluated:  10
Time elapsed:  406.01944494247437


In [53]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [172]:
n=15
symbol='T'

model = RandomForestClassifier
RandomForestClassifier()
bsearch_svm = BayesianSearch(
    search_space(model),
    model,
    X_train[symbol][[f'{x}_{n}' for x in ti_list]], 
    y_train[symbol][f'signal_{n}'], 
    X_test[symbol][[f'{x}_{n}' for x in ti_list]],
    y_test[symbol][f'signal_{n}'],
    num_eval=20
)

100%|██████████| 20/20 [00:54<00:00,  2.73s/it, best loss: -0.6072727272727273]
##### Results #####
Score best parameters:  -0.6072727272727273
Best parameters:  {'bootstrap': False, 'criterion': 'entropy', 'cv': 'btscv', 'max_depth': 56, 'max_features': 2, 'min_samples_leaf': 19, 'model': <class 'sklearn.ensemble._forest.RandomForestClassifier'>, 'n_estimators': 8}
Test Score:  0.5689948892674617
Parameter combinations evaluated:  20
Time elapsed:  54.68887519836426


## Running on all 50 stocks

In [158]:
best_params = {'bootstrap': False, 'criterion': 'gini', 'max_depth': 218, 'max_features': 1, 'min_samples_leaf': 19, 'n_estimators': 423}
model_2a = RandomForestClassifier(n_jobs=-1, **best_params)
avg, _, _ = avg_model(  
    symbol_list,
    forecast_horizon,                                 
    input_window_size,                                  
    X_train,    
    X_test,    
    y_train,    
    y_test,    
    prices_test,
    model=model_2a,
    silent = False
)

'Leg [50/50]; Elapsed Time 2475.0s\n'

,3,5,7,10,15,20,25,30
1,0.507666,0.504838,0.504157,0.505792,0.504872,0.507462,0.506099,0.504055
3,0.511073,0.508245,0.510903,0.506133,0.507632,0.507802,0.508177,0.504974
5,0.514378,0.511652,0.511141,0.513629,0.516082,0.516695,0.51891,0.516082
7,0.516082,0.516491,0.516593,0.515639,0.519625,0.520307,0.519216,0.521193
10,0.517172,0.515945,0.515264,0.513254,0.518228,0.519932,0.519387,0.517581
15,0.524497,0.520852,0.522044,0.522215,0.524259,0.531687,0.525894,0.522964
20,0.52678,0.527325,0.529063,0.527802,0.529744,0.525213,0.521942,0.521772
25,0.528382,0.527121,0.530801,0.535128,0.532164,0.524872,0.523952,0.526269
30,0.535945,0.537922,0.537206,0.537956,0.538024,0.531857,0.532913,0.535196


In [159]:
avg

,3,5,7,10,15,20,25,30
1,0.413629,0.41339,0.411721,0.412368,0.409506,0.413663,0.414037,0.412129
3,0.406065,0.406814,0.409165,0.407836,0.406951,0.410971,0.411959,0.414514
5,0.402078,0.405486,0.406917,0.405077,0.408893,0.407939,0.406337,0.408382
7,0.40644,0.406746,0.40586,0.402351,0.406678,0.40954,0.406303,0.413799
10,0.407836,0.407359,0.407019,0.40385,0.408041,0.409915,0.408825,0.405451
15,0.402589,0.408484,0.400818,0.402249,0.405554,0.406269,0.39816,0.397683
20,0.396491,0.398705,0.398637,0.394719,0.397922,0.397002,0.39925,0.396559
25,0.397615,0.394617,0.398126,0.396934,0.400443,0.401193,0.399761,0.396899
30,0.396048,0.394991,0.395707,0.392811,0.397683,0.406269,0.401704,0.398535
